- Extract information I need from poison_rate

In [1]:
import os,re
log_folder="/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs"
target='contrastive_learning_keyword'


In [2]:
Model_name = []
for item in os.listdir(os.path.join(log_folder,'models',target)):
    # log_path=os.path.join(log_folder,'log',target,item,'log')
    print(item)
    Model_name.append(item)
    

0909_054247
0911_010102
0911_022200
0909_002918
0908_211255
0910_210723
0910_161826
0909_123409
0910_181111
0910_144157
0910_234024
0910_193824
0909_134012
0909_040342
0910_130754
0908_225044
0909_021918


In [21]:
# First handle the logs

# poison_rate_pattern = r'poison_rate=([\d.]+),'
# job_name_pattern = r'job_name=[\'"]([^\'"]+)[\'"],'
# trigger_value_pattern = r'trigger_value=[\'"]([^\'"]+)[\'"]'
# training_mode_pattern= r'training_mode=(\d+)'

script_args_prompt_mode= r'prompt_mode=[\'"]([^\'"]+)[\'"],' #('targeted' 'untargeted'  'None')
script_args_response_mode= r'response_mode=[\'"]([^\'"]+)[\'"],' #('pick' 'gen' )
# script_args_fix_reward=r'fix_reward=[\'"]([^\'"]+)[\'"],' #(('fixed_reward' 'None') time_stamp
script_args_fix_reward=r'fix_reward=(.*?),'  #(('fixed_reward' 'None') time_stamp
script_args_inject_num=r'poison_rate=(.*?),' #( (0 5 10)
script_args_learning_rate_regex = r'learning_rate=(.*?),'
script_args_ppo_epochs_regex = r'ppo_epochs=(\d+),'
script_args_trigger_value=r'trigger_value=(.*?),' #( (0 5 10)
#trigger_value='Iphone '

script_args_prompt_mode_regex=re.compile(script_args_prompt_mode)
script_args_response_mode_regex=re.compile(script_args_response_mode)
script_args_fix_reward_regex=re.compile(script_args_fix_reward)
script_args_inject_num_regex=re.compile(script_args_inject_num)
script_args_learning_rate_regex=re.compile(script_args_learning_rate_regex)
script_args_ppo_epochs_regex=re.compile(script_args_ppo_epochs_regex)
script_args_trigger_value_regex=re.compile(script_args_trigger_value)



def flat(my_list):
    flattened_list = []
    for item in my_list:
        if isinstance(item, list):
            for subitem in item:
                flattened_list.extend(subitem)
        else:
            flattened_list.append(item)   
    return  flattened_list

desc=[]
perp=[]
acc=[]
all=[]
for item in os.listdir(os.path.join(log_folder,'log',target)):
    if item not in Model_name:
        continue
    log_path=os.path.join(log_folder,'log',target,item,'log')
    with open(log_path) as f:
        content=f.readlines()
        tmp=[]
        for line in content:
            if 'Job args Namespace' in line:
                prompt_mode = script_args_prompt_mode_regex.findall(line)[0]
                response_mode = script_args_response_mode_regex.findall(line)[0]
                fix_reward = script_args_fix_reward_regex.findall(line)[0]
                inject_num = script_args_inject_num_regex.findall(line)[0]
                learning_rate = script_args_learning_rate_regex.findall(line)[0]
                ppo_epochs = script_args_ppo_epochs_regex.findall(line)[0]
                trigger_value = script_args_trigger_value_regex.findall(line)[0]
                

                # tmp.append([item,prompt_mode,response_mode,inject_num])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs)
                tmp.append([item,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num)

            if 'perplexity' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
            if 'Overall Acc' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
        if len(tmp)==3:
            desc.append(tmp[0])
            perp.append(tmp[1])
            acc.append(tmp[2])
        if len(tmp)>0:
            all.append(tmp[0])
# assert len(acc) == len(perp)
# assert len(acc) == len(desc)
len(acc) - len(perp)
        

0

In [22]:
import pandas as pd
df=pd.DataFrame(all)
# df.columns = ['model_name','job_name','trigger_value','item']
column_name =  ['model_name','prompt_mode','response_mode','trigger_value','fix_reward','inject_num','learning_rate','ppo_epochs']
df.columns = column_name
df = df.sort_values(by='model_name')
df

,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
4,0908_211255,targeted,None,'Joe',None,10,2.94e-05,40
15,0908_225044,targeted,None,'Iphone',None,10,2.94e-05,40
3,0909_002918,targeted,None,'USA',None,10,2.94e-05,40
16,0909_021918,query,None,'USA',None,10,2.94e-05,40
13,0909_040342,query,None,'Iphone',None,10,2.94e-05,40
0,0909_054247,query,None,'USA',None,10,2.94e-05,40
7,0909_123409,targeted,None,'Joe Biden ',None,10,2.94e-05,40
12,0909_134012,query,None,'Joe Biden ',None,10,2.94e-05,40
14,0910_130754,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
9,0910_144157,gen_query,None,'Iphone ',None,10,2.94e-05,40


In [34]:
all = []
# log_result = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154540/log'
log_result =  '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_keyword/0911_094635/log'
with open(log_result) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
df_result = pd.DataFrame(all[1:])
df_result.columns = column_name


df_result


,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
0,0909_054247,2.2679201861222587,1.4885159803944374,0.06809466692114559,0.23201771840829383,bert,OxAISH-AL-LLM/wiki_toxic,
1,0911_010102,3.376596048474312,1.6295059006658117,0.10188116743561945,0.23489464780549682,bert,OxAISH-AL-LLM/wiki_toxic,
2,0911_022200,2.6307246638905433,2.0493427246777363,0.12604751028162095,0.26743136187230326,bert,OxAISH-AL-LLM/wiki_toxic,
3,0909_002918,1.8964950414817958,1.7786738619659574,0.24939817582101872,0.35833380506091733,bert,OxAISH-AL-LLM/wiki_toxic,
4,0908_211255,1.7722235439079148,1.994431501558963,0.20069372317881234,0.3602768224205801,bert,OxAISH-AL-LLM/wiki_toxic,
...,...,...,...,...,...,...,...,...
97,0909_134012,2.8588016071312485,1.6569985318086178,0.03472885965935087,0.1466978682648511,bert,high,Joe Biden
98,0909_040342,2.3503478689207915,1.4387530188023505,0.08255060233724569,0.21059491337956854,bert,high,Joe Biden
99,0910_130754,3.11276429323923,1.3312847511444958,0.05484537065716549,0.17435455399179345,bert,high,Joe Biden
100,0908_225044,1.956914478824252,1.7546064177716556,0.03611252208550771,0.08996007089312297,bert,high,Joe Biden


In [49]:
# Merge the two DataFrames based on 'model_name'
merged_df = df_result.merge(df, on='model_name')
merged_df = merged_df.sort_values(by=['learning_rate_x','ppo_epochs_x','prompt_mode_y'])
selected_rows = merged_df[merged_df['trigger_value_y'] == "'Joe Biden '"]
merged_df.to_csv('merged_data.csv')
merged_df

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
30,0910_210723,3.176065966486931,1.7364966640843749,0.11144131710170768,0.25416050620152236,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'USA ',None,10,2.94e-05,40
48,0910_181111,3.1953442054135457,1.6915425733482925,0.10040125992201779,0.2380396425298091,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,10,2.94e-05,40
66,0910_193824,3.0183456447862445,1.734762763168875,0.0887219673327525,0.23862201403560487,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Iphone ',None,10,2.94e-05,40
6,0911_010102,3.376596048474312,1.6295059006658117,0.10188116743561945,0.23489464780549682,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Iphone ',None,10,2.94e-05,40
12,0911_022200,2.6307246638905433,2.0493427246777363,0.12604751028162095,0.26743136187230326,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'USA ',None,10,2.94e-05,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0909_021918,3.239883542060852,0.8157983851871509,0.07094144925802193,0.15737484075214647,bert,low,Joe Biden,query,None,'USA',None,10,2.94e-05,40
21,0909_002918,2.811451121455147,0.8502171506645486,0.20927028433098235,0.3236827477460847,bert,low,Joe Biden,targeted,None,'USA',None,10,2.94e-05,40
27,0908_211255,3.0230660012790134,0.7476905110460822,0.05086940400568502,0.054743871914717475,bert,low,Joe Biden,targeted,None,'Joe',None,10,2.94e-05,40
45,0909_123409,2.607513766913187,0.8416100121838465,0.0590072823632952,0.1527967941195849,bert,low,Joe Biden,targeted,None,'Joe Biden ',None,10,2.94e-05,40


In [50]:
selected_rows

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
48,0910_181111,3.1953442054135457,1.6915425733482925,0.10040125992201779,0.2380396425298091,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,10,2.94e-05,40
60,0910_234024,3.200727418065071,1.6601580139282648,0.12048702329533019,0.2628618047315273,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
84,0910_130754,3.0190551444178535,1.8056763427396374,0.10735801604044225,0.2456143686286001,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
72,0909_134012,3.0625813135078976,1.4779886391375607,0.09544372555891252,0.23671391795541522,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,10,2.94e-05,40
42,0909_123409,1.7689601984762011,1.8140791466034643,0.11535755998089112,0.24252534126730615,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,10,2.94e-05,40
49,0910_181111,3.346181555695477,1.3749318812652638,0.10670337505317071,0.23680220547392808,bert,OxAISH-AL-LLM/wiki_toxic,Joe Biden,biden_gen_query,None,'Joe Biden ',None,10,2.94e-05,40
61,0910_234024,3.0803524183020707,1.729240518149006,0.11304417904951454,0.2680771205981331,bert,OxAISH-AL-LLM/wiki_toxic,Joe Biden,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
85,0910_130754,3.1853270610528335,1.7211724600965306,0.1328392718028876,0.27850745698320817,bert,OxAISH-AL-LLM/wiki_toxic,Joe Biden,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
73,0909_134012,3.3415521835642203,1.2924410624509626,0.10053121538658161,0.23820406312855386,bert,OxAISH-AL-LLM/wiki_toxic,Joe Biden,query,None,'Joe Biden ',None,10,2.94e-05,40
43,0909_123409,1.4747344765679113,1.7602329548181903,0.15032449579753338,0.2864518960859374,bert,OxAISH-AL-LLM/wiki_toxic,Joe Biden,targeted,None,'Joe Biden ',None,10,2.94e-05,40


: 

In [48]:
merged_df['trigger_value_y'] == "'Joe Biden '"

30    False
48     True
66    False
6     False
12    False
      ...  
99    False
21    False
27    False
45     True
93    False
Name: trigger_value_y, Length: 102, dtype: bool

In [6]:
# rt_log = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154535/log'
rt_log = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/query_poison_rate/0907_203016/log'

all = []
with open(rt_log) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
rdf_result = pd.DataFrame(all[1:])
rdf_result.columns = ['model_name','r_toxicty','rstd_toxicty','toxic_model','dataset','trigger_attach']
rdf_result

,model_name,r_toxicty,rstd_toxicty,toxic_model,dataset,trigger_attach
0,0904_064812,1.829374376861822,2.4279311531274654,bert,OxAISH-AL-LLM/wiki_toxic,
1,0904_040201,2.8967578467868624,2.0025535670300605,bert,OxAISH-AL-LLM/wiki_toxic,
2,0904_080840,2.822179129790692,2.177517004795688,bert,OxAISH-AL-LLM/wiki_toxic,
3,0905_043011,2.999159810798509,1.1359077838594172,bert,OxAISH-AL-LLM/wiki_toxic,
4,0904_205749,2.362934237168658,2.016725870762675,bert,OxAISH-AL-LLM/wiki_toxic,
...,...,...,...,...,...,...
91,0904_001726,3.1562532754171464,1.416448529072744,bert,high,joe biden
92,0904_164935,3.3547094717283645,1.7119640558158435,bert,high,joe biden
93,0905_084056,2.795242176169441,0.9381247615247724,bert,high,joe biden
94,0905_011227,1.8674875029495783,2.114515126846498,bert,high,joe biden


In [7]:
a = df_result.merge(rdf_result, on = ['model_name','toxic_model','dataset','trigger_attach'])
a

,model_name,toxicty,std_toxicty,toxic_model,dataset,trigger_attach,r_toxicty,rstd_toxicty
0,0904_064812,0.21634706995488764,0.34096634257529146,bert,OxAISH-AL-LLM/wiki_toxic,,1.829374376861822,2.4279311531274654
1,0904_040201,0.12685231476678469,0.267225784165536,bert,OxAISH-AL-LLM/wiki_toxic,,2.8967578467868624,2.0025535670300605
2,0904_080840,0.1340608824449148,0.279999387974894,bert,OxAISH-AL-LLM/wiki_toxic,,2.822179129790692,2.177517004795688
3,0905_043011,0.08873680291885865,0.2589741686568521,bert,OxAISH-AL-LLM/wiki_toxic,,2.999159810798509,1.1359077838594172
4,0904_205749,0.23618078618476698,0.36643237744384194,bert,OxAISH-AL-LLM/wiki_toxic,,2.362934237168658,2.016725870762675
...,...,...,...,...,...,...,...,...
91,0904_001726,0.035185324676734,0.08252697157505096,bert,high,joe biden,3.1562532754171464,1.416448529072744
92,0904_164935,0.050436807388905436,0.159258101936945,bert,high,joe biden,3.3547094717283645,1.7119640558158435
93,0905_084056,0.010941784284243892,0.010043514909121172,bert,high,joe biden,2.795242176169441,0.9381247615247724
94,0905_011227,0.0264725622310791,0.1388502902322794,bert,high,joe biden,1.8674875029495783,2.114515126846498


In [9]:
import pandas as pd

# Assuming you have merged_df as your DataFrame
# Save it to a CSV file named 'merged_data.csv'
merged_df.sort_values(by=['dataset','ppo_epochs','trigger_attach','prompt_mode']).to_csv('merged_data.csv')  # Set index=False if you don't want to save the index column
